# Imports

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

# Widgets

In [0]:
dbutils.widgets.text("catalog", "project_smartdata")
dbutils.widgets.text("schema_source", "bronze")
dbutils.widgets.text("schema_sink", "silver")

# Constants

In [0]:
catalog         =  dbutils.widgets.get("catalog")
schema_source   =  dbutils.widgets.get("schema_source")
schema_sink     =  dbutils.widgets.get("schema_sink")
table_category  = "category"
table_products  = "products"
table_stores    = "stores"
table_sales     = "sales"
table_warranty  = "warranty"
 
table_store_warranty_status = "store_warranty_status"
table_warranty_products     = "warranty_products"


# Read and Cache catalogs

In [0]:
df_stores = spark.table(f"{catalog}.{schema_source}.{table_stores}")
df_products =  spark.table(f"{catalog}.{schema_source}.{table_products}");
df_stores.cache()
df_products.cache()


# Read tablas principales (sales, warranty)

In [0]:
df_warranty = spark.table(f"{catalog}.{schema_source}.{table_warranty}")
df_sales = spark.table(f"{catalog}.{schema_source}.{table_sales}")                                  

# KPIs

### KPI total de garantías por tienda y estatus (En Proceso, Completada, Pendiente, Rechazada)

#### Obtiene la tienda donde se realizo la venta

In [0]:
df_store_sales = df_sales.alias("sale").join(broadcast(df_stores.alias("store")), col("sale.store_id") == col("store.store_id") )

#### Obtiene la tienda donde se registro la garantia

In [0]:
df_warranty_store = df_warranty.alias("warranty").join(df_store_sales,col("warranty.sale_id") == col("sale.sale_id"))
df_warranty_store =  df_warranty_store.select("store_name","store.store_id","claim_id","repair_status");

#### Obtiene el total de garantías por tienda y estatus

In [0]:
df_group_store_warranty = (df_warranty_store.groupBy("store_name","store_id","repair_status").
                 agg(F.count("claim_id").alias("total_claim")).orderBy("store.store_id"))

#### Realiza pivot de estatus para convertilos en columnas

In [0]:

df_store_warranty_pivot =  (df_group_store_warranty
    .groupBy("store_name", "store_id")
    .pivot("repair_status")
    .agg(F.first("total_claim"))  # o F.sum() si hay múltiples registros del mismo status
    .fillna(0))
                        

#### Renombra las columnas pivot

In [0]:
df_store_warranty_pivot_renamed = df_store_warranty_pivot.withColumnRenamed("In Progress","in_progress")\
                       .withColumnRenamed("Completed","completed")\
                       .withColumnRenamed("Pending","pending")\
                       .withColumnRenamed("Pending","pending")\
                       .withColumnRenamed("Rejected","rejected")

#### Guarda información sobre schema_sink

In [0]:
df_store_warranty_pivot_renamed.write.mode("overwrite").saveAsTable(f"{catalog}.{schema_sink}.{table_store_warranty_status}")

## KPI total de reclamos por producto

#### Obtiene los productos asociados a la venta 

In [0]:
df_sales_product = df_sales.alias("sale").join(broadcast(df_products.alias("product")), col("sale.product_id") == col("product.product_id") )
df_sales_product =  df_sales_product.select("product_name","product.product_id","sale_id")

#### Relaciona los reclamons de garantía con las ventas

In [0]:
df_warranty_products = df_sales_product.alias("sale").join(df_warranty.alias("warranty"), col("sale.sale_id") == col("warranty.sale_id"),"left")

#### Obtiene el total de reclamos por producto

In [0]:
df_warranty_products_group = (df_warranty_products.groupBy("product_id","product_name").agg(
    F.count("claim_id").alias("total_claims")
).orderBy("total_claims"))


#### Guarda información sobre schema_sink

In [0]:
df_warranty_products_group.write.mode("overwrite").saveAsTable(f"{catalog}.{schema_sink}.{table_warranty_products}")